In [7]:
from lxml import html
import requests
import pandas as pd
import os
from selenium import webdriver
import time

In [8]:
def data_to_csv(data, filename):
    df = pd.DataFrame.from_dict(data).reset_index().set_index('index')
    print(df)
    writer = open(os.path.join(filename), "a")#, encoding="utf-8")
    csv_data = df.to_csv()
    writer.write('\n')
    writer.write(csv_data)
    writer.close()

In [27]:
url = 'http://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;opposition=1;opposition=2;opposition=25;opposition=3;opposition=4;opposition=40;opposition=5;opposition=6;opposition=7;opposition=8;orderby=start;page='
ext = ';result=1;result=2;spanmax3=03+Jun+2019;spanmin3=03+Jul+2015;spanval3=span;team=1;team=2;team=25;team=3;team=4;team=40;team=5;team=6;team=7;team=8;template=results;type=team;view=innings'

for i in range(1,13):
    page = requests.get(url+str(i)+ext)
    tree = html.fromstring(page.content)
    team = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "engineTable", " " )) and (((count(preceding-sibling::*) + 1) = 5) and parent::*)]//*[contains(concat( " ", @class, " " ), concat( " ", "left", " " )) and (((count(preceding-sibling::*) + 1) = 1) and parent::*)]//text()')
    score = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "engineTable", " " )) and (((count(preceding-sibling::*) + 1) = 5) and parent::*)]//tr//*[(((count(preceding-sibling::*) + 1) = 2) and parent::*)]//text()')
    overs = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "engineTable", " " )) and (((count(preceding-sibling::*) + 1) = 5) and parent::*)]//tr//*[(((count(preceding-sibling::*) + 1) = 3) and parent::*)]//text()')
    RPO = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "engineTable", " " )) and (((count(preceding-sibling::*) + 1) = 5) and parent::*)]//tr//*[(((count(preceding-sibling::*) + 1) = 4) and parent::*)]//text()')
    Inns = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "engineTable", " " )) and (((count(preceding-sibling::*) + 1) = 5) and parent::*)]//tr//*[(((count(preceding-sibling::*) + 1) = 5) and parent::*)]//text()')
    Result = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "engineTable", " " )) and (((count(preceding-sibling::*) + 1) = 5) and parent::*)]//tr//*[(((count(preceding-sibling::*) + 1) = 6) and parent::*)]//text()')
    Opp = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "left", " " )) and (((count(preceding-sibling::*) + 1) = 8) and parent::*)]//text()')
    Opp = Opp[0::2]
    Date = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "data1", " " ))]//b//text()')
    Date = ['Date']+ Date

    data_dict= {
        'Team': team,
        'Score': score,
        'Overs': overs,
        'RPO': RPO,
        'Innings': Inns,
        'Result': Result,
        'Opposition': Opp,
        'Date': Date
    }

    filename= f"cricketstats.csv"
    data_to_csv(data_dict, filename)

In [118]:
df = pd.read_csv('cricketstats.csv',sep=',')
df.set_index('num',inplace=True)

In [119]:
df1 = df.iloc[0:596:2, :]
df2 = df.iloc[1:596:2, :]

In [120]:
df1.reset_index(inplace=True, drop=True)
df2.reset_index(inplace=True, drop=True)

In [126]:
df_col = pd.concat([df1,df2], axis=1)

In [128]:
df_col.to_csv('Final_Dataset.csv',index=False) 

Dataset was cleaned to remove extra columns and rename columns to avoid confusion

result - 0 = loss, 1 = win

Team1 batted first and Team2 batted second